# 06 SMH

See: http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf

In [1]:
import pandas as pd
import numpy as np
from tensorflow.contrib import keras

from smh_eyetracking.features02 import config as config_features02
from smh_eyetracking.features02.utils.features02_dlib import FACE, JAWLINE, NOSE, LEFT_EYE, RIGHT_EYE, TARGETS
from smh_eyetracking.keras import config as config_keras
from smh_eyetracking.keras import losses
from smh_eyetracking.utils import data_model

## Load data

In [2]:
data, imgs_left, imgs_right = data_model.load(
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_CSV,
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

## Split data

In [3]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    validation_size=0.15,
    test_size=0.15
)

In [4]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

Train length: 65742
Validation length: 383
Test length: 447


## Model

### Architecture

In [70]:
img_width, img_height = config_features02.FEATURES02_EYES_SIZE
img_shape = (img_height, img_width)

def eye_prediction(imgs, eye_features, other_features):
    c1 = keras.layers.Conv2D(
        filters=16,
        kernel_size=(5,5),
        strides=(1,1),
        padding="valid",
        activation='relu'
    )(keras.layers.Reshape((24,32,1))(imgs))
    s2 = keras.layers.AveragePooling2D(
        pool_size=(2, 2),
        strides=None,  # Same as pool_size
        padding='valid'
    )(c1)
    c3 = keras.layers.Conv2D(
        filters=32,
        kernel_size=(3,3),
        strides=(1,1),
        padding="valid",
        activation='relu'
    )(s2)
    s4 = keras.layers.AveragePooling2D(
        pool_size=(2, 2),
        strides=None,  # Same as pool_size
        padding='valid'
    )(c3)
    c5 = keras.layers.Conv2D(
        filters=64,
        kernel_size=(3,3),
        strides=(1,1),
        padding="valid",
        activation='relu'
    )(s4)
    f4 = keras.layers.Dense(128, activation="relu")(keras.layers.Flatten()(c5))
    #f6 = keras.layers.Dense(64, activation="relu")(f4)
    c7 = keras.layers.Concatenate()([f4, eye_features])
    f8 = keras.layers.Dense(32, activation="relu")(c7)
    #f9 = keras.layers.Dense(32, activation="relu")(f4)
    return f8
    

def get_model():

    # Inputs
    left_imgs = keras.layers.Input(shape=img_shape, name='left_imgs', dtype='float32')
    right_imgs = keras.layers.Input(shape=img_shape, name='right_imgs', dtype='float32')
    
    features_jawline = keras.layers.Input(shape=(len(JAWLINE),), name='features_jawline', dtype='float32')
    features_nose = keras.layers.Input(shape=(len(NOSE),), name='features_nose', dtype='float32')
    features_left_eye = keras.layers.Input(shape=(len(LEFT_EYE),), name='features_left_eye', dtype='float32')
    features_right_eye = keras.layers.Input(shape=(len(RIGHT_EYE),), name='features_right_eye', dtype='float32')
    features_face = keras.layers.Input(shape=(len(FACE),), name='features_face', dtype='float32')
    
    other_features = keras.layers.Concatenate()([features_jawline, features_nose, features_face])
    

    eye_predictions_x_01 = keras.layers.Dense(64, activation="relu")(keras.layers.Concatenate()([
        eye_prediction(left_imgs, features_left_eye, other_features),
        eye_prediction(right_imgs, features_right_eye, other_features),
        other_features
    ]))
    eye_predictions_x = keras.layers.Dense(1, activation="linear")(eye_predictions_x_01)
    

    eye_predictions_y_01 = keras.layers.Dense(64, activation="relu")(keras.layers.Concatenate()([
        eye_prediction(left_imgs, features_left_eye, other_features),
        eye_prediction(right_imgs, features_right_eye, other_features),
        other_features
    ]))
    eye_predictions_y = keras.layers.Dense(1, activation="linear")(eye_predictions_y_01)
    
    
    eye_predictions = keras.layers.Concatenate()([eye_predictions_x,eye_predictions_y])
   
    # Model
    model = keras.models.Model(
        inputs=[
            left_imgs, right_imgs,
            features_jawline, features_nose, features_left_eye, features_right_eye, features_face
        ],
        outputs=[eye_predictions]
    )
    return model
    
    

### Parameters

In [71]:
MODEL_NAME = '06_smh-01'

EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 0.0005
DECAY = 0.000001
DROPOUT = 0

LOSS = losses.mean_euclidean

In [72]:
model = get_model()


model.compile(
    loss=LOSS,
    metrics=[losses.mean_euclidean],
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=DECAY)
)

print("Parameters to adjust: {}".format(
    np.sum([keras.backend.count_params(p) for p in set(model.trainable_weights)])
))

Parameters to adjust: 390530


### Train

In [73]:
model.fit(
    x={
        'left_imgs':train_imgs_left,
        'right_imgs': train_imgs_right,
        'features_jawline': train_data[JAWLINE].as_matrix(),
        'features_nose': train_data[NOSE].as_matrix(),
        'features_left_eye': train_data[LEFT_EYE].as_matrix(),
        'features_right_eye': train_data[RIGHT_EYE].as_matrix(),
        'features_face': train_data[FACE].as_matrix()
    },
    y=train_data[TARGETS].as_matrix(),
    validation_data=(
        {
            'left_imgs': validation_imgs_left,
            'right_imgs': validation_imgs_right,
            'features_jawline': validation_data[JAWLINE].as_matrix(),
            'features_nose': validation_data[NOSE].as_matrix(),
            'features_left_eye': validation_data[LEFT_EYE].as_matrix(),
            'features_right_eye': validation_data[RIGHT_EYE].as_matrix(),
            'features_face': validation_data[FACE].as_matrix()
        },
        validation_data[TARGETS].as_matrix()
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1, callbacks=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0
)

Train on 65742 samples, validate on 383 samples
Epoch 1/20
65742/65742 [==============================] - 22s - loss: 0.3573 - mean_euclidean: 0.3573 - val_loss: 0.2864 - val_mean_euclidean: 0.2864
Epoch 2/20
65742/65742 [==============================] - 21s - loss: 0.2565 - mean_euclidean: 0.2565 - val_loss: 0.2334 - val_mean_euclidean: 0.2334
Epoch 3/20
65742/65742 [==============================] - 21s - loss: 0.2220 - mean_euclidean: 0.2220 - val_loss: 0.2323 - val_mean_euclidean: 0.2323
Epoch 4/20
65742/65742 [==============================] - 21s - loss: 0.2028 - mean_euclidean: 0.2028 - val_loss: 0.2067 - val_mean_euclidean: 0.2067
Epoch 5/20
65742/65742 [==============================] - 21s - loss: 0.1872 - mean_euclidean: 0.1872 - val_loss: 0.2037 - val_mean_euclidean: 0.2037
Epoch 6/20
65742/65742 [==============================] - 21s - loss: 0.1763 - mean_euclidean: 0.1763 - val_loss: 0.1967 - val_mean_euclidean: 0.1967
Epoch 7/20
65742/65742 [============================

KeyboardInterrupt: 

In [ ]:
model.save(config_keras.PATH_MODELS_KERAS+MODEL_NAME)

### Test

In [ ]:
model_test = keras.models.load_model(
    filepath=config_keras.PATH_MODELS_KERAS+MODEL_NAME,
    custom_objects={
        "mean_euclidean": losses.mean_euclidean,
        "ms_euclidean": losses.ms_euclidean,
        "reg_mean_euclidean": losses.reg_mean_euclidean
    }
)

In [ ]:
model.evaluate(
    x={
        'left_imgs':test_imgs_left,
        'right_imgs': test_imgs_right,
        'features_jawline': test_data[JAWLINE].as_matrix(),
        'features_nose': test_data[NOSE].as_matrix(),
        'features_left_eye': test_data[LEFT_EYE].as_matrix(),
        'features_right_eye': test_data[RIGHT_EYE].as_matrix(),
        'features_face': test_data[FACE].as_matrix()
    },
    y=test_data[TARGETS].as_matrix(),
    batch_size=1,
    verbose=1, sample_weight=None
)